This is all unnecessary and didn't end up using it. 

In [1]:
import pandas as pd
import geopandas as gpd

In [3]:
folder = "C:\\Users\\z3258367\\OneDrive - UNSW\\#PhD\\"
meshblocks = pd.read_csv(''.join(folder + "Data\\ABS Data\\2016_NSW_MBs\\2016 census mesh block counts.csv"))
#DZNs = pd.read_csv(''.join(folder + "Melbourne Data\\2016 Victoria DZN employment.csv"), dtype='int64')
mb_shapes = gpd.read_file(''.join(folder + "Data\\ABS Data\\2016_NSW_MBs\\MB_2016_NSW.shp"))

In [7]:
meshblocks.groupby('MB_CATEGORY_NAME_2016')['Person'].describe()

,count,mean,std,min,25%,50%,75%,max
MB_CATEGORY_NAME_2016,,,,,,,,
ANTARCTICA,1.0,0.000000,NaN,0.0,0.0,0.0,0.0,0.0
Commercial,15430.0,13.993778,60.686012,0.0,0.0,3.0,11.0,3026.0
Education,8266.0,7.608396,72.219615,0.0,0.0,0.0,0.0,2883.0
Hospital/Medical,1178.0,46.271647,56.209820,0.0,4.0,24.0,74.0,468.0
Industrial,7724.0,3.631150,15.944396,0.0,0.0,0.0,3.0,520.0
MIGRATORY,11.0,0.454545,1.507557,0.0,0.0,0.0,0.0,5.0
NOUSUALRESIDENCE,9.0,5018.888889,4859.779585,55.0,912.0,2863.0,7565.0,12510.0
OFFSHORE,9.0,40.888889,108.627166,0.0,0.0,4.0,8.0,330.0
Other,6223.0,62.387112,54.094424,0.0,17.0,54.0,96.0,986.0


Using all meshblocks with population.

In [141]:
employ_mbs = meshblocks[meshblocks['MB_CATEGORY_NAME_2016'].isin(
    ['Commercial','Primary Production','Hospital/Medical','Education','Other','Industrial'])]

Next sum the meshblock areas by DZN code. Ie, the output is a list of DZNs along with summed areas for each of them, of the area of the employment meshbocks within. This is then joined to the DZN Place of Work numbers data.

In [120]:
employ_areas = pd.DataFrame(employ_mbs.groupby('DZN_CODE_2016')['AREA_ALBERS_SQKM'].sum())

In [121]:
DZN_areas = DZNs.join(employ_areas, on='DZN (POW)', how='left')

This is the portion of jobs we lose with this method - jobs that are in DZNs that are made of entirely excluded meshblocks (residential, transport, parkland, water). For states I have done so far it's under 5% so considered it acceptable. One potential improvement would be to manually change some meshblock categories, for example an airport from 'transport' to 'industrial'. (Most transport meshblocks are just road or rail corridors so are better excluded).

In [122]:
DZN_areas[(DZN_areas['Number']>0) & (DZN_areas['AREA_ALBERS_SQKM'].isna())]['Number'].sum()/DZN_areas['Number'].sum()

0.03846591307284983

The DZN 'Job Density' is the number of people who report that DZN as their place of work, divided by the area of employment meshblocks within. This density is then used to calculate the job number for each of those meshblocks.

In [123]:
DZN_areas['JobDensity'] = DZN_areas['Number']/DZN_areas['AREA_ALBERS_SQKM']

employ_mbs = employ_mbs.join(DZN_areas.set_index('DZN (POW)'), on='DZN_CODE_2016', how='inner', rsuffix='_DZN')

employ_mbs['Jobs'] = employ_mbs['JobDensity']*employ_mbs['AREA_ALBERS_SQKM']

The employment figures are attached to the meshblock shapefiles, and centroids are also output, as currently I am using the centroids as the points for walkability calculations.

In [133]:
mb_shapes['MB_CODE16'] = mb_shapes['MB_CODE16'].astype('int64')
employ_mbs['MB_CODE_2016'] = employ_mbs['MB_CODE_2016'].astype('int64')

employ_shapes = mb_shapes.join(employ_mbs.set_index('MB_CODE_2016')[['DZN_CODE_2016','Jobs']], how='right', on='MB_CODE16')

In [138]:
employ_shapes.to_file(''.join(folder + "Melbourne Data\\Vic_Employment_meshblocks.gpkg"), layer='meshblocks')

centroids = employ_shapes.copy()
centroids.geometry = (centroids.geometry
                         .to_crs('EPSG:7856')
                         .centroid)
centroids.to_file(''.join(folder + "Melbourne Data\\Vic_Employment_meshblocks.gpkg"), layer='centroids')